# Extreme Value Analysis

## Introduction

Extreme Value Analysis (EVA) is a branch of statistics that deals with the extreme deviations from the median of probability distributions. It's particularly important in fields like hydrology, meteorology, and finance where rare but extreme events (floods, storms, market crashes) can have significant impacts.

This notebook demonstrates how to use the `statista` package to perform extreme value analysis on time series data. We'll explore two common distributions used in EVA:

1. **Gumbel Distribution** - A special case of the Generalized Extreme Value (GEV) distribution with shape parameter = 0
2. **Generalized Extreme Value (GEV) Distribution** - A family of continuous probability distributions developed within extreme value theory

## Setup and Data Loading

First, let's import the necessary libraries and set up our environment:


In [ ]:
import matplotlib
%matplotlib inline
import pandas as pd
from statista.distributions import ConfidenceInterval, PlottingPosition, Distributions


### Loading Time Series Data

We'll work with two different time series datasets for our analysis. These datasets might represent annual maximum values of some variable (like rainfall, river discharge, etc.).


In [ ]:
# Load two time series datasets from text files
time_series1 = pd.read_csv("../../../examples/data/time_series1.txt", header=None)[0].tolist()
time_series2 = pd.read_csv("../../../examples/data/time_series2.txt", header=None)[0].tolist()

# Display the first time series to understand the data we're working with
print(time_series1)

# Gumbel Distribution

The Gumbel distribution is commonly used to model the distribution of maximum values. It's particularly useful for modeling extreme events like floods, maximum wind speeds, or earthquake magnitudes. The distribution is characterized by two parameters:

- **Location parameter (μ)**: Controls the mode of the distribution
- **Scale parameter (β)**: Controls the dispersion of the distribution

## Parameter Estimation Methods

There are several methods to estimate the parameters of a Gumbel distribution. We'll explore two common approaches:

### Maximum-Likelihood Method

The Maximum-Likelihood Estimation (MLE) method finds parameter values that maximize the likelihood of observing the given data. It's a widely used statistical method that often provides efficient estimators.


In [ ]:
# Create a Gumbel distribution object with our first time series
gumbel_series_1 = Distributions("Gumbel", time_series1)

# Fit the model using Maximum-Likelihood Estimation (MLE)
# This estimates the location and scale parameters of the Gumbel distribution
param_mle_series_1 = gumbel_series_1.fit_model(method="mle")

# Perform goodness-of-fit tests to evaluate how well the distribution fits our data
# Kolmogorov-Smirnov test
gumbel_series_1.ks()
# Chi-square test
gumbel_series_1.chisquare()

# Display the estimated parameters
print(param_mle_series_1)


#### Visualizing the Fitted Distribution

After fitting the distribution, it's important to visualize how well it represents our data. We'll look at both the probability density function (PDF) and cumulative distribution function (CDF).

##### Probability Density Function (PDF)

The PDF shows the relative likelihood of the random variable taking a specific value. The area under the curve between two points gives the probability of the random variable falling within that range.


In [ ]:
# Calculate and plot the probability density function (PDF)
pdf, fig, ax = gumbel_series_1.pdf(plot_figure=True)


##### Cumulative Distribution Function (CDF)

The CDF gives the probability that the random variable takes a value less than or equal to a given point. It's particularly useful for determining probabilities of extreme events.


In [ ]:
# Calculate and plot the cumulative distribution function (CDF)
cdf, _, _ = gumbel_series_1.cdf(plot_figure=True)

### L-moments Method

L-moments are an alternative to conventional moments and provide a more robust way to estimate distribution parameters, especially for small sample sizes. They are less sensitive to outliers and perform well for extreme value distributions.


In [ ]:
# Fit the Gumbel distribution using the L-moments method
param_lmoments_series_1 = gumbel_series_1.fit_model(method="lmoments")

# Perform goodness-of-fit tests to evaluate how well this new fit performs
# Kolmogorov-Smirnov test
gumbel_series_1.ks()
# Chi-square test
gumbel_series_1.chisquare()

# Display the estimated parameters
print(param_lmoments_series_1)


#### Comparing Visualization Results

Let's visualize the distribution fitted using L-moments to compare with our previous MLE results.

##### Probability Density Function (PDF) with L-moments


In [ ]:
# Calculate and plot the PDF using parameters estimated with L-moments
pdf, fig, ax = gumbel_series_1.pdf(plot_figure=True)


##### Cumulative Distribution Function (CDF) with L-moments


In [ ]:
# Calculate and plot the CDF using parameters estimated with L-moments
cdf, fig, ax = gumbel_series_1.cdf(plot_figure=True)


### Confidence Intervals

Confidence intervals provide a range of values that likely contain the true parameter value with a specified confidence level. They help quantify the uncertainty in our parameter estimates.


In [ ]:
# Calculate the confidence interval with 90% confidence level (alpha=0.1)
# This helps us understand the uncertainty in our estimates
upper, lower, fig, ax = gumbel_series_1.confidence_interval(alpha=0.1, plot_figure=True)


### Combined PDF and CDF Plot

For a comprehensive view, we can generate a combined plot showing both the PDF and CDF of our fitted distribution.


In [ ]:
# Generate a combined plot showing both PDF and CDF
fig, ax = gumbel_series_1.plot()

## Truncated Distribution: Focusing on Extreme Values

In extreme value analysis, we're often most interested in the tail of the distribution (the extreme values). By using a threshold, we can focus our analysis specifically on values above a certain level, which can lead to better modeling of extreme events.

This approach is particularly useful when:
- Only the extreme values are of interest (e.g., flood levels above a critical threshold)
- The full dataset might not follow the theoretical distribution, but the extremes do
- You want to improve the fit specifically for the tail of the distribution


In [ ]:
# Import the specific Gumbel class for more advanced operations
from statista.distributions import Gumbel

# Set a high threshold to focus only on values above 18
threshold = 18

# Fit the model using optimization method with a truncated distribution objective function
# This will only consider values above the threshold when fitting the distribution
gev_param_mle_series_1 = gumbel_series_1.fit_model(
    method="optimization", 
    obj_func=Gumbel.truncated_distribution, 
    threshold=threshold
)

# Display the estimated parameters
print(gev_param_mle_series_1)

# Visualize the results
gumbel_series_1.plot()
gumbel_series_1.confidence_interval(plot_figure=True)


### Effect of Different Thresholds

The choice of threshold can significantly impact the parameter estimates. Let's try a lower threshold to see how it affects our results.


In [ ]:
# Set a lower threshold of 15
threshold = 15

# Fit the model again with the new threshold
gev_param_mle_series_1 = gumbel_series_1.fit_model(
    method="optimization", 
    obj_func=Gumbel.truncated_distribution, 
    threshold=threshold
)

# Display the new estimated parameters
print(gev_param_mle_series_1)

# Visualize the results with the new threshold
gumbel_series_1.plot()
gumbel_series_1.confidence_interval(plot_figure=True)

# Generalized Extreme Value (GEV) Distribution

The Generalized Extreme Value (GEV) distribution is a family of continuous probability distributions that combines three simpler distributions:
- Gumbel (Type I)
- Fréchet (Type II)
- Weibull (Type III)

The GEV distribution is characterized by three parameters:
- **Location parameter (μ)**: Controls the center of the distribution
- **Scale parameter (σ)**: Controls the width of the distribution
- **Shape parameter (ξ)**: Determines which type of extreme value distribution is represented
  - ξ = 0: Gumbel distribution
  - ξ > 0: Fréchet distribution
  - ξ < 0: Weibull distribution

The shape parameter is particularly important as it determines the behavior of the distribution's tail, which is crucial for modeling extreme events.

## Maximum-Likelihood Estimation for GEV

Let's apply the GEV distribution to our second time series dataset:


In [ ]:
# Create a GEV distribution object with our second time series
gev_series_2 = Distributions("GEV", time_series2)

# Fit the model using Maximum-Likelihood Estimation (default method)
# This estimates all three parameters of the GEV distribution
gev_param_mle_series_2 = gev_series_2.fit_model()

# Perform goodness-of-fit tests
# Kolmogorov-Smirnov test
gev_series_2.ks()
# Chi-square test
gev_series_2.chisquare()

# Display the estimated parameters (location, scale, and shape)
print(gev_param_mle_series_2)

# Calculate and plot the probability density function (PDF)
pdf, fig, ax = gev_series_2.pdf(plot_figure=True)

# Calculate and plot the cumulative distribution function (CDF)
cdf, _, _ = gev_series_2.cdf(plot_figure=True)


## L-moments Estimation for GEV

As with the Gumbel distribution, we can also use the L-moments method to estimate the parameters of the GEV distribution. This method is often more robust for small sample sizes and less sensitive to outliers.


In [ ]:
# Fit the GEV distribution using the L-moments method
gev_param_lm_series_2 = gev_series_2.fit_model(method="lmoments")

# Display the estimated parameters
print(gev_param_lm_series_2)

# Calculate and plot the PDF with L-moments parameters
pdf, fig, ax = gev_series_2.pdf(plot_figure=True)

# Calculate and plot the CDF with L-moments parameters
cdf, _, _ = gev_series_2.cdf(plot_figure=True)

## Confidence Intervals for GEV Distribution

Confidence intervals are crucial in extreme value analysis as they help quantify the uncertainty in our estimates. This is especially important when dealing with rare events where data might be limited.

### Using Plotting Positions

Plotting positions are empirical estimates of the cumulative distribution function. The Weibull plotting position is commonly used in hydrology and provides a non-parametric estimate of the probability of non-exceedance.


In [ ]:
# Import the GEV class for specific functions
from statista.distributions import GEV

# Calculate the non-exceedance probabilities using Weibull plotting position
# This gives us empirical estimates of the CDF based on the data
cdf_Weibul = PlottingPosition.weibul(time_series2)

# Calculate theoretical quantiles corresponding to the empirical probabilities
# This allows us to compare the theoretical distribution with the empirical data
qth = gev_series_2.inverse_cdf(cdf_Weibul)

# Calculate and plot confidence intervals
# alpha=0.1 means we're calculating 90% confidence intervals
upper, lower, fig, ax = gev_series_2.confidence_interval(
    prob_non_exceed=cdf_Weibul,  # Use the empirical probabilities
    alpha=0.1,                   # 90% confidence level
    n_samples=100,               # Number of bootstrap samples
    plot_figure=True             # Generate a plot
)


### Bootstrap Method for Confidence Intervals

Bootstrap is a resampling technique that allows us to estimate the sampling distribution of a statistic by resampling with replacement from the observed data. It's particularly useful when the theoretical distribution of the statistic is complicated or unknown.


In [ ]:
# Perform bootstrap to calculate confidence intervals
CI = ConfidenceInterval.boot_strap(
    time_series2,                # Our data
    gevfit=gev_param_lm_series_2, # Parameters from L-moments estimation
    n_samples=100,               # Number of bootstrap samples
    F=cdf_Weibul,                # Empirical probabilities
    method="lmoments",           # Parameter estimation method
    state_function=GEV.ci_func   # Function to calculate statistics for each bootstrap sample
)

# Extract the lower and upper bounds of the confidence interval
lower_bound = CI["lb"]
upper_bound = CI["ub"]

# Display the bounds
print("Lower bound of confidence interval:")
print(lower_bound)
print("\nUpper bound of confidence interval:")
print(upper_bound)


## Final Visualization

Let's create a comprehensive visualization of our GEV distribution fit and confidence intervals:


In [ ]:
# Generate a combined plot showing both PDF and CDF
fig, ax = gev_series_2.plot()

# Add confidence intervals to the plot
upper_bound, lower_bound, fig, ax = gev_series_2.confidence_interval(plot_figure=True)


## Conclusion

In this notebook, we've explored extreme value analysis using the `statista` package. We've covered:

1. **Gumbel Distribution** - A special case of GEV used for modeling maxima
2. **Generalized Extreme Value (GEV) Distribution** - A more flexible family of distributions
3. **Parameter Estimation Methods** - Maximum Likelihood and L-moments
4. **Truncated Distributions** - Focusing on values above a threshold
5. **Confidence Intervals** - Quantifying uncertainty in our estimates

These techniques are essential for analyzing extreme events in various fields such as hydrology, meteorology, and finance. By properly modeling extreme values, we can better understand and prepare for rare but potentially catastrophic events.
